<a href="https://colab.research.google.com/github/julioclerio/database_modeling_and_normalization/blob/main/2_consulta_sql_em_bd_postgres_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymysql

In [ ]:
# IMPORTAÇÃO BIBLIOTECA
import pymysql

In [ ]:
# CRIAR CONEXÃO AO MYSQL - AWS

try:
  conexao = pymysql.connect( user= '',
                             host= '',
                             port= ,
                             password= '',
                             database= '')
except pymysql.Error as e:
  print('Erro ao se conectar: ', e)

In [ ]:
# CRIANDO UM DATABASE PARA SOLUÇÃO DO EXERCICIO
novo_banco = 'avaliacao_part2'

In [ ]:
# CRIAR O CURSOR
cursor = conexao.cursor()

In [ ]:
criar_database = f" CREATE DATABASE {novo_banco}"

try:
    # Executar a consulta SQL para criar o banco de dados
    cursor.execute(criar_database)
    print(f"Banco de dados '{novo_banco}' criado com sucesso.")
except Exception as e:
    print(f"Erro ao criar o banco de dados: {str(e)}")

Erro ao criar o banco de dados: (1044, "Access denied for user 'clerio_fernandes'@'%' to database 'avaliacao_part2'")


In [ ]:
cursor.execute(criar_database)

In [ ]:
# SELECIONAR O BANCO DE DADOS CRIADO
selecionado_database = f"USE {novo_banco}"

try:
    cursor.execute(selecionado_database)
    print(f"Banco de dados '{novo_banco}' selecionado com sucesso.")

except Exception as e:
    print(f"Erro ao selecionar o banco de dados: {str(e)}")

In [ ]:
cursor.execute(selecionado_database)

In [ ]:
# COMANDO DDL CRIAR TABELA CLIENTE
ddl_produtos = """CREATE TABLE produtos_ex2 (
    id SERIAL PRIMARY KEY  AUTO_INCREMENT,
    nome VARCHAR(100) NOT NULL,
    preco DECIMAL(10, 2) NOT NULL
)"""

In [ ]:
cursor.execute(ddl_produtos)

In [ ]:
# COMANDO DDL CRIAR TABELA CATEGORIA
ddl_categorias = """CREATE TABLE categorias_ex2 (
    id SERIAL PRIMARY KEY  AUTO_INCREMENT,
    nome VARCHAR(100) NOT NULL
)"""

In [ ]:
cursor.execute(ddl_categorias)

0

In [ ]:
# COMANDO DDL CRIAR TABELA RELACIONAL PRODUTO CATEGORIA
ddl_produtos_categorias = """CREATE TABLE produtos_categorias_ex2 (
    produto_id INTEGER REFERENCES produtos(id),
    categoria_id INTEGER REFERENCES categorias(id),
    PRIMARY KEY (produto_id, categoria_id)
)"""

In [ ]:
cursor.execute(ddl_produtos_categorias)

0

INSERINDO DADOS NA TABELA PARA CONSULTAS

In [ ]:
# INSERINDO DADOS NA TABELA PARA REALIZAR AS CONSULTAS
import random
categorias = {
    "Frutas": ["Abacaxi", "Banana", "Maçã", "Laranja", "Manga"],
    "Eletrônicos": ["Celular", "Notebook", "Televisão", "Tablet", "Smartwatch"],
    "Vestuário": ["Camiseta", "Calça", "Vestido", "Tênis", "Chapéu"],
    "Livros": ["Romance", "Ficção", "História", "Matemática", "Ciências"],
    "Esportes": ["Bola", "Raquete", "Tênis de Corrida", "Bicicleta", "Capacete"],
    "Móveis": ["Mesa", "Cadeira", "Sofá", "Estante", "Cama"],
    "Bebidas": ["Água", "Refrigerante", "Suco", "Cerveja", "Vinho"],
    "Alimentos": ["Arroz", "Feijão", "Macarrão", "Carne", "Peixe"],
    "Brinquedos": ["Boneca", "Carrinho", "Quebra-cabeça", "Jogo de Tabuleiro", "Pelúcia"],
    "Instrumentos Musicais": ["Guitarra", "Bateria", "Piano", "Violino", "Flauta"]
}

# Inserindo categorias e produtos
for categoria, produtos in categorias.items():
    # Inserir categoria
    sql_insert_categoria = f"INSERT INTO categorias_ex2 (nome) VALUES ('{categoria}')"
    cursor.execute(sql_insert_categoria)
    id_categoria = cursor.lastrowid  # Pega o ID da última categoria inserida

    # Inserir produtos relacionados à categoria
    for produto in produtos:
        preco_produto = round(random.uniform(10, 1000), 2)  # Gera um preço aleatório entre 10 e 1000 reais
        sql_insert_produto = f"INSERT INTO produtos_ex2 (nome, preco) VALUES ('{produto}', {preco_produto})"
        cursor.execute(sql_insert_produto)
        id_produto = cursor.lastrowid  # Pega o ID do último produto inserido

        # Relacionar produto e categoria
        sql_insert_relacao = f"INSERT INTO produtos_categorias_ex2 (produto_id, categoria_id) VALUES ({id_produto}, {id_categoria})"
        cursor.execute(sql_insert_relacao)

conexao.commit()


print("Categorias e produtos inseridos com sucesso!")

CONSULTAS AO BANCO SQL

In [ ]:
# 3) Liste os nomes de todos os produtos que custam mais de 100 reais,
# ordenando-os primeiramente pelo preço e em segundo lugar pelo nome.
# Use alias para mostrar o nome da coluna nome como "Produto" e da coluna preco como
# "Valor". A resposta da consulta não deve mostrar outras colunas de dados.
try:
    cursor = conexao.cursor()
    sql_query = """
    SELECT nome AS Produto, preco AS Valor
    FROM produtos_ex2
    WHERE preco > 100
    ORDER BY preco DESC, nome ASC
    """
    cursor.execute(sql_query)
    df_consulta100 = pd.read_sql_query(sql_query, conexao)
    print(df_consulta100.to_markdown(index = False))

except Exception as e:
    print(f"Erro ao executar a consulta: {e}")

| Produto           |   Valor |
|:------------------|--------:|
| Sofá              |  985.9  |
| Carne             |  984.28 |
| Flauta            |  983.34 |
| Violino           |  980.48 |
| História          |  973.23 |
| Estante           |  971.03 |
| Arroz             |  948.09 |
| Laranja           |  890.99 |
| Bateria           |  846.8  |
| Bola              |  821.46 |
| Água              |  798.91 |
| Pelúcia           |  771.37 |
| Maçã              |  739.61 |
| Tênis             |  695.62 |
| Tênis de Corrida  |  688.35 |
| Jogo de Tabuleiro |  680.4  |
| Boneca            |  659.99 |
| Cerveja           |  591.06 |
| Calça             |  564.07 |
| Macarrão          |  551.96 |
| Mesa              |  532.94 |
| Notebook          |  510.79 |
| Banana            |  505.28 |
| Raquete           |  441.81 |
| Piano             |  422.43 |
| Cadeira           |  395.17 |
| Matemática        |  385.94 |
| Vestido           |  351.44 |
| Feijão            |  333.89 |
| Capace

In [ ]:
# 4) Liste todos os ids e preços de produtos cujo preço seja maior do
# que a média de todos os preços encontrados na tabela "produtos".

media_preco = """SELECT AVG(preco) AS media_preco FROM produtos_ex2"""
df_media = pd.read_sql_query(media_preco, conexao)
print(df_media.to_markdown(index = False))

|   media_preco |
|--------------:|
|       464.356 |


In [ ]:
try:
    sql_query_maior_media = """
    SELECT id AS Ids, preco
    FROM produtos_ex2
    WHERE preco > (SELECT AVG(preco) FROM produtos_ex2)
    ORDER BY preco DESC
    """
    cursor.execute(sql_query_maior_media)
    df_maior_media = pd.read_sql_query(sql_query_maior_media, conexao)
    print(df_maior_media.to_markdown(index = False))

except Exception as e:
    print(f"Erro ao executar a consulta: {e}")

|   Ids |   preco |
|------:|--------:|
|    31 |  985.9  |
|    42 |  984.28 |
|    53 |  983.34 |
|    52 |  980.48 |
|    21 |  973.23 |
|    32 |  971.03 |
|    39 |  948.09 |
|     7 |  890.99 |
|    50 |  846.8  |
|    24 |  821.46 |
|    34 |  798.91 |
|    48 |  771.37 |
|     6 |  739.61 |
|    17 |  695.62 |
|    26 |  688.35 |
|    47 |  680.4  |
|    44 |  659.99 |
|    37 |  591.06 |
|    15 |  564.07 |
|    41 |  551.96 |
|    29 |  532.94 |
|    10 |  510.79 |
|     5 |  505.28 |


In [ ]:
# 5) Para cada categoria, mostre o preço médio do conjunto de
#produtos a ela associados. Caso uma categoria não tenha nenhum
#produto a ela associada, esta categoria não deve aparecer no
#resultado final. A consulta deve estar ordenada pelos nomes das
#categorias.

try:
    sql_query_categoria = """
    SELECT c.nome AS categoria, AVG(p.preco) AS media_preco
    FROM produtos_ex2 AS p
    JOIN produtos_categorias_ex2 AS pc ON p.id = pc.produto_id
    JOIN categorias_ex2 AS c ON pc.categoria_id = c.id
    GROUP BY c.nome
    """
    cursor.execute(sql_query_categoria)
    df_categoria = pd.read_sql_query(sql_query_categoria, conexao)
    print(df_categoria.to_markdown(index = False))

except Exception as e:
    print(f"Erro ao executar a consulta: {e}")


| categoria             |   media_preco |
|:----------------------|--------------:|
| Frutas                |       509.574 |
| Eletrônicos           |       305.906 |
| Vestuário             |       425.012 |
| Livros                |       348.568 |
| Esportes              |       485.782 |
| Móveis                |       585.044 |
| Bebidas               |       376.812 |
| Alimentos             |       602.92  |
| Brinquedos            |       471.076 |
| Instrumentos Musicais |       712.81  |


CRIANDO UM MENU PARA INSERIR NOVOS PRODUTOS

In [ ]:
import warnings
import pandas as pd
from IPython.display import display, Markdown

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# LEITURA DOS ARQUIVOS PARA ANALISE
def exibir_markdown(texto_md):
    display(Markdown(texto_md))

In [ ]:
def inserir_categoria():
  nova_categoria = input('Digite a nova categoria que deseja cadastrar: ')
  sql_insert_categoria = "INSERT INTO categorias_ex2 (nome) values ('{}')".format(nova_categoria)
  cursor.execute(sql_insert_categoria)
  conexao.commit()
  print('\n ------------ CATEGORIA CADASTRADA COM SUCESSO ------------------------')

# MOSTRAR A CATEGORIA ADICIONADA
  df_categoria = pd.read_sql_query('select * from categorias_ex2', conexao)
  print(df_categoria.to_markdown(index = False))

In [ ]:
def inserir_produto():
  novo_produto = input("Digite o produto para cadastrar: ")
  preco = float(input(f"Digite o preço do {novo_produto}: "))

  df_categoria = pd.read_sql_query('select * from categorias_ex2', conexao)
  print(df_categoria.to_markdown(index=False))

  id_categoria = int(input(f'O produto {novo_produto} pertence a qual categoria? 0 para nenhuma categoria'))


  if id_categoria == 0:
    sql_insert_produto = f"insert into produtos_ex2 (nome, preco) values ('{novo_produto}',{preco})"
  else:
    sql_insert_produto = f"insert into produtos_ex2 ( nome, preco ) values ('{novo_produto}', {preco})"
    sql_associate_category = f"insert into produtos_categorias_ex2 (produto_id, categoria_id) values (LAST_INSERT_ID(), {id_categoria})"
    cursor.execute(sql_associate_category)

  cursor.execute(sql_insert_produto)
  conexao.commit()

# MOSTRAR A TABELA COMPLETA COM O PRODUTO ADICIONADO

  print('\n ------------ PRODUTO CADASTRADO COM SUCESSO! ------------------------')
  df_produto = pd.read_sql_query("""select p.id as id_produto, p.nome as produto, c.nome as categoria
                                      from produtos p
                                      left join produtos_categorias pc on p.id = pc.produto_id
                                      left join categorias c on pc.categoria_id = c.id""", conexao)
  print(df_produto.to_markdown(index=False))

In [ ]:
# RENOMEAR NOME DO PRODUTO
def alterar_produto():
  df_produto = pd.read_sql_query('select * from produto', conexao)
  exibir_markdown(df_produto.to_markdown(index=False))

  id_produto = int(input('Digite o ID do produto a ser editada: '))
  nome_novo_produto = input("Digite o novo valor do produto: ")
  preco_novo_produto = float(input("Digite o novo valor do produto: "))

  sql_alterar_produto = f"""update produto
                          set nome = '{nome_novo_produto}',
                          set preco = {preco_novo_produto}
                          where id = {id_produto} """

  cursor.execute(sql_alterar_produto)
  conexao.commit()

# EXIBIR O PRODUTO

  print('\n ------------ UPDATE EFETUADO COM SUCESSO! ------------------------')
  df_produto = pd.read_sql_query('select * from produto', conexao)
  print(df_produto.to_markdown(index=False))



In [ ]:
# RENOMEAR CATEGORIA
def alterar_categoria():
  df_categoria = pd.read_sql_query('select * from categorias_ex2', conexao)
  exibir_markdown(df_categoria.to_markdown(index=False))

  id_categoria_update = int(input("Digite o ID da categoria para modificar: "))
  nome_novo_categoria = input("Digite o novo nome da categoria: ")
  sql_alterar_categoria = f"""update categorias_ex2
                          set nome = '{nome_novo_categoria}'
                          where id = {categoria_id} """
  cursor.execute(sql_alterar_categoria)
  conexao.commit()

  print(f"CATEGORIA ATUALIZADA COM SUCESSO.")

  df_categoria = pd.read_sql_query('select * from categoria', conexao)
  print(df_categoria.to_markdown(index=False))

In [ ]:
cursor = conexao.cursor()
while True:
  opcao_menu = input("""
  selecione a opção desejada:
  1- cadastrar categoria
  2- cadastrar produto
  3- alterar categoria
  4- alterar produto
  5- sair
  """)

  if opcao_menu == '1':
    print('cadastrar categoria')
    inserir_categoria()
  elif opcao_menu == '2':
    print('cadastrar produto')
    inserir_produto()
  elif opcao_menu == '3':
    print('alterar categoria')
    alterar_categoria()
  elif opcao_menu == '4':
    print('alterar produto')
    alterar_produto()
  elif opcao_menu == '5':
    print('Até Mais')
    cursor.close()
    break